# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV, RepeatedKFold

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dhaneswaramandrasat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dhaneswaramandrasat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dhaneswaramandrasat/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df =  pd.read_sql("SELECT * FROM messages", engine)
X = df['message'].values
y = df.drop(['id','message','original','genre'], axis=1).values

In [5]:
np.where(y > 1)

(array([], dtype=int64), array([], dtype=int64))

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    
    text = re.sub(r"[^a-zA-Z0-9]"," ", text.lower())
    
    text = word_tokenize(text)
    
    tokens = [WordNetLemmatizer().lemmatize(w) for w in text if w not in stopwords.words("english")]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [32]:

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))  
])

In [115]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x171938940>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=None,
                  

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [116]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [117]:
y_pred = pipeline.predict(X_test)

In [127]:
label = list(df.drop(['id','message','original','genre'], axis=1).columns)
classification_report(y_test, y_pred, output_dict=True, target_names=label)

{'related': {'precision': 0.8419432842382941,
  'recall': 0.9442800788954635,
  'f1-score': 0.8901801278326554,
  'support': 4056},
 'request': {'precision': 0.793046357615894,
  'recall': 0.5020964360587002,
  'f1-score': 0.6148908857509628,
  'support': 954},
 'offer': {'precision': 0.1951219512195122,
  'recall': 0.13793103448275862,
  'f1-score': 0.16161616161616163,
  'support': 58},
 'aid_related': {'precision': 0.7450980392156863,
  'recall': 0.7114155251141553,
  'f1-score': 0.727867320719458,
  'support': 2190},
 'medical_help': {'precision': 0.4117647058823529,
  'recall': 0.09130434782608696,
  'f1-score': 0.1494661921708185,
  'support': 460},
 'medical_products': {'precision': 0.4588235294117647,
  'recall': 0.1270358306188925,
  'f1-score': 0.1989795918367347,
  'support': 307},
 'search_and_rescue': {'precision': 0.3230769230769231,
  'recall': 0.10552763819095477,
  'f1-score': 0.1590909090909091,
  'support': 199},
 'security': {'precision': 0.19607843137254902,
  'rec

### 6. Improve your model
Use grid search to find better parameters. 

In [145]:
parameters = { 'clf__estimator__n_estimators': [10],
    'vect__ngram_range': [(1, 1)]}

cv = GridSearchCV(pipeline, param_grid = parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [146]:
cv.fit(X_train, y_train)

In [147]:
y_pred = cv.predict(X_test)
classification_report(y_test, y_pred, output_dict=True, target_names=label)

/Users/dhaneswaramandrasat/anaconda3/envs/dsnd/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/dhaneswaramandrasat/anaconda3/envs/dsnd/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'related': {'precision': 0.8482758620689655,
  'recall': 0.9097633136094675,
  'f1-score': 0.8779443254817988,
  'support': 4056},
 'request': {'precision': 0.7839851024208566,
  'recall': 0.4412997903563941,
  'f1-score': 0.5647216633132126,
  'support': 954},
 'offer': {'precision': 0.22448979591836735,
  'recall': 0.1896551724137931,
  'f1-score': 0.205607476635514,
  'support': 58},
 'aid_related': {'precision': 0.7291330102315563,
  'recall': 0.6182648401826484,
  'f1-score': 0.6691376328144304,
  'support': 2190},
 'medical_help': {'precision': 0.48717948717948717,
  'recall': 0.12391304347826088,
  'f1-score': 0.1975736568457539,
  'support': 460},
 'medical_products': {'precision': 0.6142857142857143,
  'recall': 0.14006514657980457,
  'f1-score': 0.22811671087533156,
  'support': 307},
 'search_and_rescue': {'precision': 0.5333333333333333,
  'recall': 0.08040201005025126,
  'f1-score': 0.13973799126637554,
  'support': 199},
 'security': {'precision': 0.18032786885245902,
  

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [4]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(word_tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [34]:
pipeline = Pipeline([
    ('features', FeatureUnion([
    ('text_pipeline', Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer())
        ])),
        ('starting_verb', StartingVerbExtractor())
    ])),
    ('clf', MultiOutputClassifier(MultinomialNB())) 
    ])

parameters = [
    {
    "clf": [MultiOutputClassifier(RandomForestClassifier())],
    'clf__estimator__n_estimators': [10],
    'features__text_pipeline__vect__ngram_range': [(1, 1)]
    },
    {
    'clf': [MultiOutputClassifier(MultinomialNB())],
    'clf__estimator__alpha': [0.5, 1.5],
    'features__text_pipeline__vect__ngram_range': [(1, 1)]
    },
    {
    "clf": [MultiOutputClassifier(LogisticRegression())],
    "clf__estimator__estimator__C": [0.01, 0.1],
    'features__text_pipeline__vect__ngram_range': [(1, 1)]
    }]


cv = GridSearchCV(pipeline, param_grid = parameters)

In [5]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        
        ('text_pipeline', Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer())
        ])),
        
        ('starting_verb', StartingVerbExtractor())
    ])),
    
    ('clf', MultiOutputClassifier(RandomForestClassifier()))  
    ])

parameters = { 'clf__estimator__n_estimators': [10],
    'features__text_pipeline__vect__ngram_range': [(1, 1)]}



cv = GridSearchCV(pipeline, param_grid = parameters)



In [7]:
pipeline = Pipeline([

        
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),    
    ('clf', MultiOutputClassifier(MultinomialNB()))  
    ])

parameters = { 'clf__estimator__alpha': [0.5, 1.5],
    'vect__ngram_range': [(1, 1)]}



cv = GridSearchCV(pipeline, param_grid = parameters)



In [21]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        
        ('text_pipeline', Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer())
        ])),
        
        ('starting_verb', StartingVerbExtractor())
    ])),
    
    ('clf', MultiOutputClassifier(LogisticRegression()))
    ])

parameters = {'clf__estimator__estimator__C': [0.01, 0.1],
    'clf__estimator__estimator__penalty': ["l1", "l2"],
     'features__text_pipeline__vect__ngram_range': [(1, 1)]}


cv = GridSearchCV(pipeline, param_grid = parameters)

In [ ]:
# Naive Bayes with verb extractor

pipeline = Pipeline([
    ('features', FeatureUnion([
        
        ('text_pipeline', Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer())
        ])),
        
        ('starting_verb', StartingVerbExtractor())
    ])),
    
    ('clf', MultiOutputClassifier(MultinomialNB()))
    ])

parameters = { 'clf__estimator__alpha': [0.5, 1.5],
    'features__text_pipeline__vect__ngram_range': [(1, 1)]}




cv = GridSearchCV(pipeline, param_grid = parameters)


In [22]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'features', 'clf', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__verbose', 'features__text_pipeline', 'features__starting_verb', 'features__text_pipeline__memory', 'features__text_pipeline__steps', 'features__text_pipeline__verbose', 'features__text_pipeline__vect', 'features__text_pipeline__tfidf', 'features__text_pipeline__vect__analyzer', 'features__text_pipeline__vect__binary', 'features__text_pipeline__vect__decode_error', 'features__text_pipeline__vect__dtype', 'features__text_pipeline__vect__encoding', 'features__text_pipeline__vect__input', 'features__text_pipeline__vect__lowercase', 'features__text_pipeline__vect__max_df', 'features__text_pipeline__vect__max_features', 'features__text_pipeline__vect__min_df', 'features__text_pipeline__vect__ngram_range', 'features__text_pipeline__vect__preprocessor', 'features__text_pipeline__vect__stop_words', 'features__text_pipeline__vect__strip_accents

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)
label = list(df.drop(['id','message','original','genre'], axis=1).columns)
classification_report(y_test, y_pred, output_dict=True, target_names=label)

/Users/dhaneswaramandrasat/anaconda3/envs/dsnd/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/dhaneswaramandrasat/anaconda3/envs/dsnd/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'related': {'precision': 0.7948303715670436,
  'recall': 0.9867134620205565,
  'f1-score': 0.8804384297058494,
  'support': 3989},
 'request': {'precision': 0.8317307692307693,
  'recall': 0.3802197802197802,
  'f1-score': 0.521870286576169,
  'support': 910},
 'offer': {'precision': 0.6666666666666666,
  'recall': 0.03389830508474576,
  'f1-score': 0.06451612903225806,
  'support': 59},
 'aid_related': {'precision': 0.7231320368474923,
  'recall': 0.6370604147880974,
  'f1-score': 0.677372962607862,
  'support': 2218},
 'medical_help': {'precision': 0.5925925925925926,
  'recall': 0.035555555555555556,
  'f1-score': 0.06708595387840671,
  'support': 450},
 'medical_products': {'precision': 0.5,
  'recall': 0.020761245674740483,
  'f1-score': 0.03986710963455149,
  'support': 289},
 'search_and_rescue': {'precision': 0.5,
  'recall': 0.017241379310344827,
  'f1-score': 0.03333333333333333,
  'support': 174},
 'security': {'precision': 0.6,
  'recall': 0.02654867256637168,
  'f1-score'

In [5]:
class LengthExtractor(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_length = pd.Series(X).apply(len)
        return pd.DataFrame(X_length)

In [10]:
pipeline = Pipeline([
    ('features', FeatureUnion([
    ('text_pipeline', Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer())
        ])),
        ('starting_verb', StartingVerbExtractor()),
        ('length_extractor', LengthExtractor())
    ])),
    ('clf', MultiOutputClassifier(MultinomialNB())) 
    ])

parameters = [
    {
    "clf": [MultiOutputClassifier(RandomForestClassifier())],
    'clf__estimator__n_estimators': [10],
    'features__text_pipeline__vect__ngram_range': [(1, 1)]
    },
    {
    'clf': [MultiOutputClassifier(MultinomialNB())],
    'clf__estimator__alpha': [0.5, 1.5],
    'features__text_pipeline__vect__ngram_range': [(1, 1)]
    }]


cv = GridSearchCV(pipeline, param_grid = parameters, scoring=["f1_weighted", "f1_micro", "f1_samples"],
        refit="f1_weighted")

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)
label = list(df.drop(['id','message','original','genre'], axis=1).columns)
classification_report(y_test, y_pred, output_dict=True, target_names=label)

/Users/dhaneswaramandrasat/anaconda3/envs/dsnd/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1514: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/Users/dhaneswaramandrasat/anaconda3/envs/dsnd/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1514: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/Users/dhaneswaramandrasat/anaconda3/envs/dsnd/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1514: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/Users/dhaneswaramandrasat/anaconda3/envs/dsnd/lib/python3.8/site-packages/sklearn/metrics/_classificat

{'related': {'precision': 0.8445349626612356,
  'recall': 0.9246778989098117,
  'f1-score': 0.8827912477823772,
  'support': 4036},
 'request': {'precision': 0.8116760828625236,
  'recall': 0.44849115504682624,
  'f1-score': 0.5777479892761394,
  'support': 961},
 'offer': {'precision': 1.0,
  'recall': 0.109375,
  'f1-score': 0.19718309859154928,
  'support': 64},
 'aid_related': {'precision': 0.7350054525627044,
  'recall': 0.6127272727272727,
  'f1-score': 0.6683192860684184,
  'support': 2200},
 'medical_help': {'precision': 0.5825242718446602,
  'recall': 0.13157894736842105,
  'f1-score': 0.2146690518783542,
  'support': 456},
 'medical_products': {'precision': 0.7254901960784313,
  'recall': 0.10979228486646884,
  'f1-score': 0.1907216494845361,
  'support': 337},
 'search_and_rescue': {'precision': 0.6086956521739131,
  'recall': 0.07567567567567568,
  'f1-score': 0.13461538461538464,
  'support': 185},
 'security': {'precision': 0.5384615384615384,
  'recall': 0.04861111111111

In [12]:
cv.best_score_

0.5230761875295227

In [15]:
cv.best_estimator_.get_params()['clf']

MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                       ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
                                                       max_samples=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                                       min_weight_fraction_leaf=0.0,
                  

In [16]:
df = pd.DataFrame.from_dict(cv.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf,param_clf__estimator__n_estimators,param_features__text_pipeline__vect__ngram_range,param_clf__estimator__alpha,params,split0_test_f1_weighted,...,std_test_f1_micro,rank_test_f1_micro,split0_test_f1_samples,split1_test_f1_samples,split2_test_f1_samples,split3_test_f1_samples,split4_test_f1_samples,mean_test_f1_samples,std_test_f1_samples,rank_test_f1_samples
0,139.052664,9.283140,27.515644,0.597453,MultiOutputClassifier(estimator=RandomForestCl...,10,"(1, 1)",NaN,{'clf': MultiOutputClassifier(estimator=Random...,0.530956,...,0.009289,1,0.489770,0.481471,0.486679,0.484372,0.472507,0.482960,0.005894,1
1,96.379920,0.444683,24.550847,0.482630,MultiOutputClassifier(estimator=MultinomialNB(...,NaN,"(1, 1)",0.5,{'clf': MultiOutputClassifier(estimator=Multin...,0.383818,...,0.006029,2,0.471657,0.469461,0.475397,0.472116,0.464206,0.470567,0.003704,2
2,96.753494,1.117447,24.814322,1.328778,MultiOutputClassifier(estimator=MultinomialNB(...,NaN,"(1, 1)",1.5,{'clf': MultiOutputClassifier(estimator=Multin...,0.303140,...,0.005896,3,0.449307,0.450265,0.452437,0.450112,0.438883,0.448201,0.004773,3


In [17]:
df["mean_test_f1_weighted"]

0    0.523076
1    0.384808
2    0.302147
Name: mean_test_f1_weighted, dtype: float64

In [18]:
df["mean_test_f1_micro"]

0    0.590459
1    0.509360
2    0.455127
Name: mean_test_f1_micro, dtype: float64

In [19]:
df["mean_test_f1_samples"]

0    0.482960
1    0.470567
2    0.448201
Name: mean_test_f1_samples, dtype: float64

In [7]:
pipeline = Pipeline([
    ('features', FeatureUnion([
    ('text_pipeline', Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer())
        ])),
        ('starting_verb', StartingVerbExtractor()),
        ('length_extractor', LengthExtractor())
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier())) 
])

In [22]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'features', 'clf', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__verbose', 'features__text_pipeline', 'features__starting_verb', 'features__length_extractor', 'features__text_pipeline__memory', 'features__text_pipeline__steps', 'features__text_pipeline__verbose', 'features__text_pipeline__vect', 'features__text_pipeline__tfidf', 'features__text_pipeline__vect__analyzer', 'features__text_pipeline__vect__binary', 'features__text_pipeline__vect__decode_error', 'features__text_pipeline__vect__dtype', 'features__text_pipeline__vect__encoding', 'features__text_pipeline__vect__input', 'features__text_pipeline__vect__lowercase', 'features__text_pipeline__vect__max_df', 'features__text_pipeline__vect__max_features', 'features__text_pipeline__vect__min_df', 'features__text_pipeline__vect__ngram_range', 'features__text_pipeline__vect__preprocessor', 'features__text_pipeline__vect__stop_words', 'features__text

In [8]:
parameters = {
    'clf__estimator__n_estimators': [10, 50],
    "clf__estimator__max_depth": [5, 10],
    'clf__estimator__min_samples_split': [2, 4],
    'features__text_pipeline__tfidf__use_idf': (True, False),
    'features__text_pipeline__vect__ngram_range': [(1, 1)],
    'features__transformer_weights': (
            {'text_pipeline': 1, 'starting_verb': 0.5,'length_extractor':0.25},
            {'text_pipeline': 0.8, 'starting_verb': 1,'length_extractor':0.5},
    )
}

cv = GridSearchCV(pipeline, param_grid = parameters, scoring=["f1_weighted", "f1_micro", "f1_samples"],
        refit="f1_weighted")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)
label = list(df.drop(['id','message','original','genre'], axis=1).columns)
classification_report(y_test, y_pred, output_dict=True, target_names=label)

In [6]:
pipeline = Pipeline([
    ('features', FeatureUnion([
    ('text_pipeline', Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer())
        ])),
        ('starting_verb', StartingVerbExtractor()),
        ('length_extractor', LengthExtractor())
    ])),
    ('clf', MultiOutputClassifier(LinearSVC(dual=False))) 
])

In [9]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'features', 'clf', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__verbose', 'features__text_pipeline', 'features__starting_verb', 'features__length_extractor', 'features__text_pipeline__memory', 'features__text_pipeline__steps', 'features__text_pipeline__verbose', 'features__text_pipeline__vect', 'features__text_pipeline__tfidf', 'features__text_pipeline__vect__analyzer', 'features__text_pipeline__vect__binary', 'features__text_pipeline__vect__decode_error', 'features__text_pipeline__vect__dtype', 'features__text_pipeline__vect__encoding', 'features__text_pipeline__vect__input', 'features__text_pipeline__vect__lowercase', 'features__text_pipeline__vect__max_df', 'features__text_pipeline__vect__max_features', 'features__text_pipeline__vect__min_df', 'features__text_pipeline__vect__ngram_range', 'features__text_pipeline__vect__preprocessor', 'features__text_pipeline__vect__stop_words', 'features__text

In [7]:
parameters = {
    "clf__estimator__C": [1, 5, 10],
    "features__text_pipeline__tfidf__smooth_idf":[True,False],
    'features__text_pipeline__vect__ngram_range': [(1, 1)]
}

cv = GridSearchCV(pipeline, param_grid = parameters, scoring=["f1_weighted", "f1_micro", "f1_samples"],
        refit="f1_weighted")

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)
label = list(df.drop(['id','message','original','genre'], axis=1).columns)
classification_report(y_test, y_pred, output_dict=True, target_names=label)

/Users/dhaneswaramandrasat/anaconda3/envs/dsnd/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1514: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/Users/dhaneswaramandrasat/anaconda3/envs/dsnd/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1514: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/Users/dhaneswaramandrasat/anaconda3/envs/dsnd/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1514: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/Users/dhaneswaramandrasat/anaconda3/envs/dsnd/lib/python3.8/site-packages/sklearn/metrics/_classificat

{'related': {'precision': 0.8662435354960037,
  'recall': 0.9189526184538653,
  'f1-score': 0.8918199419167473,
  'support': 4010},
 'request': {'precision': 0.7510431154381085,
  'recall': 0.576307363927428,
  'f1-score': 0.6521739130434783,
  'support': 937},
 'offer': {'precision': 0.625,
  'recall': 0.09259259259259259,
  'f1-score': 0.16129032258064516,
  'support': 54},
 'aid_related': {'precision': 0.7245480494766888,
  'recall': 0.6882060551287844,
  'f1-score': 0.705909617612978,
  'support': 2213},
 'medical_help': {'precision': 0.5294117647058824,
  'recall': 0.28832951945080093,
  'f1-score': 0.37333333333333335,
  'support': 437},
 'medical_products': {'precision': 0.6229508196721312,
  'recall': 0.2638888888888889,
  'f1-score': 0.37073170731707317,
  'support': 288},
 'search_and_rescue': {'precision': 0.5319148936170213,
  'recall': 0.14619883040935672,
  'f1-score': 0.22935779816513763,
  'support': 171},
 'security': {'precision': 0.46153846153846156,
  'recall': 0.05

In [36]:
from sklearn.metrics import f1_score
cv.f1_score(y_test, y_pred, labels=label)

AttributeError: 'GridSearchCV' object has no attribute 'f1_score'

In [12]:
cv.best_score_

0.6155272747140996

In [15]:
cv.best_estimator_.get_params()['clf']

MultiOutputClassifier(estimator=LinearSVC(C=1, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1,
                                          loss='squared_hinge', max_iter=1000,
                                          multi_class='ovr', penalty='l2',
                                          random_state=None, tol=0.0001,
                                          verbose=0),
                      n_jobs=None)

In [16]:
df = pd.DataFrame.from_dict(cv.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__estimator__C,param_features__text_pipeline__tfidf__smooth_idf,param_features__text_pipeline__vect__ngram_range,params,split0_test_f1_weighted,split1_test_f1_weighted,...,std_test_f1_micro,rank_test_f1_micro,split0_test_f1_samples,split1_test_f1_samples,split2_test_f1_samples,split3_test_f1_samples,split4_test_f1_samples,mean_test_f1_samples,std_test_f1_samples,rank_test_f1_samples
0,114.471961,14.345203,25.203056,1.005654,1,True,"(1, 1)","{'clf__estimator__C': 1, 'features__text_pipel...",0.609697,0.618791,...,0.004555,1,0.500850,0.500786,0.495602,0.506681,0.508956,0.502575,0.004741,1
1,109.973585,7.972863,25.061218,0.730724,1,False,"(1, 1)","{'clf__estimator__C': 1, 'features__text_pipel...",0.607746,0.618406,...,0.004699,2,0.500154,0.501331,0.496023,0.504877,0.508526,0.502182,0.004249,2
2,109.227613,1.134708,25.537145,0.974892,5,True,"(1, 1)","{'clf__estimator__C': 5, 'features__text_pipel...",0.597880,0.611394,...,0.004769,4,0.478836,0.475873,0.470917,0.474128,0.476784,0.475308,0.002669,3
3,120.984959,13.156697,27.182625,2.358343,5,False,"(1, 1)","{'clf__estimator__C': 5, 'features__text_pipel...",0.597372,0.611624,...,0.004938,3,0.473872,0.472833,0.469236,0.474536,0.483569,0.474809,0.004747,4
4,131.891182,9.023587,29.801757,2.088761,10,True,"(1, 1)","{'clf__estimator__C': 10, 'features__text_pipe...",0.589253,0.605591,...,0.005675,6,0.465532,0.471805,0.460955,0.462572,0.467519,0.465677,0.003818,6
5,123.534973,8.866063,27.424876,2.304197,10,False,"(1, 1)","{'clf__estimator__C': 10, 'features__text_pipe...",0.592406,0.602759,...,0.006300,5,0.467395,0.464765,0.462466,0.470541,0.473058,0.467645,0.003817,5


In [17]:
df["mean_test_f1_weighted"]

0    0.615527
1    0.614288
2    0.605377
3    0.605828
4    0.598721
5    0.600064
Name: mean_test_f1_weighted, dtype: float64

In [18]:
df["mean_test_f1_micro"]

0    0.651767
1    0.650982
2    0.626388
3    0.626485
4    0.616008
5    0.617639
Name: mean_test_f1_micro, dtype: float64

In [20]:
df["mean_test_f1_samples"]

0    0.502575
1    0.502182
2    0.475308
3    0.474809
4    0.465677
5    0.467645
Name: mean_test_f1_samples, dtype: float64

In [31]:
from sklearn.metrics import multilabel_confusion_matrix

In [34]:
multilabel_confusion_matrix(y_test, y_pred)

array([[[ 665,  569],
        [ 325, 3685]],

       [[4128,  179],
        [ 397,  540]],

       [[5187,    3],
        [  49,    5]],

       [[2452,  579],
        [ 690, 1523]],

       [[4695,  112],
        [ 311,  126]],

       [[4910,   46],
        [ 212,   76]],

       [[5051,   22],
        [ 146,   25]],

       [[5131,    7],
        [ 100,    6]],

       [[4989,   45],
        [ 145,   65]],

       [[5212,    3],
        [  24,    5]],

       [[4799,   75],
        [ 171,  199]],

       [[4582,  107],
        [ 168,  387]],

       [[4662,   89],
        [ 209,  284]],

       [[5125,   15],
        [  72,   32]],

       [[5076,   23],
        [ 118,   27]],

       [[5148,    3],
        [  83,   10]],

       [[4986,   27],
        [ 185,   46]],

       [[4903,   45],
        [ 169,  127]],

       [[4351,  177],
        [ 569,  147]],

       [[4821,   50],
        [ 342,   31]],

       [[4923,   33],
        [ 235,   53]],

       [[4921,   50],
        [ 19

### 9. Export your model as a pickle file

In [37]:
import pickle
filename = 'model_final.pkl'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

Visualization

In [24]:
import plotly.graph_objs as go

In [48]:
y = df.groupby('genre').count()['message'].reset_index().message.tolist()

In [49]:
x = df.groupby('genre').count()['message'].reset_index().genre.tolist()

In [60]:
x

['direct', 'news', 'social']

In [63]:
graph_one = []
graph_one.append(
go.Bar(
x = x,
y = y
    )
)

layout_one = dict(
title = "Genres Count",
    xaxis = dict(title = 'Genres',),
    yaxis = dict(title = 'Count')
)



In [64]:
fig = go.Figure(data=graph_one, layout=layout_one)
fig.show()

In [26]:
category_count = df.drop(['id', 'message','original','genre'], axis=1).sum(axis=0).reset_index()
category_count.columns = ['category', 'count']
category_count = category_count.sort_values('count')
y = category_count.category.tolist()
x = category_count['count'].tolist()

In [27]:
graph_two = []
graph_two.append(
go.Bar(
x = x,
y = y,
orientation='h'
    )
)

layout_two = dict(
title = "Categories Count",
    xaxis = dict(title = 'Count',),
    yaxis = dict(title = 'Categories')
)

fig = go.Figure(data=graph_two, layout=layout_two)
fig.show()